The notebook is for pipeline all data preparation of the football match data

In [30]:
import re
import pandas as pd
import os
import numpy as np
from csv import reader
import plotly.express as px
import missingno as msno

# load pickle and read content
import pickle
d = pickle.load(open('./ELO/elo_dict.pkl', 'rb'))

Functions to get total goals so far for home team and away team in each game. The dataframe will call the apply() so that it will loop all records in the dataframe. And it will filter all records which round < current round and seperated with home and away for each team. The filter records then can calculate home total goals and away total goals

In [31]:
def getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, team, round):
    # determine home or away and get the score 
    # get home game of the team
    home_pd = data[(data["League"]==league) & (data["Home_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_home_score_sofar =  home_pd['Result'].str.extract(r'(\d)-\d')
    home_total_score = df_home_score_sofar[0].astype('Int64').sum()

    # get away game of the team
    away_pd = data[(data["League"]==league) & (data["Away_Team"]==team) & (data["Season"]==season) & (data["Round"]<round)]
    df_away_score_sofar =  away_pd['Result'].str.extract(r'\d-(\d)')
    away_total_score = df_away_score_sofar[0].astype('Int64').sum()

    # calculate total goals
    return (home_total_score + away_total_score)


def fillWithTotalGoalSoFar(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, hteam, round)
    away_goal_so_far = getLeagueSeasonTeamBeforeRoundTotalGoal(data, league, season, ateam, round)

    return [home_goal_so_far, away_goal_so_far]

Function to get elo home and away for each record in the dataframe by apply()

In [32]:
def fillWithELO(link):
    if link not in d:
        return [pd.NA, pd.NA]
    else:
        return [d[link]['Elo_home'], d[link]['Elo_away']]

Function to get recent performance with apply() similarly

In [33]:
def findRecentPreviousRounds(currentRound, limit):
    if currentRound<=limit:
        return None
    else:
        r = []
        for l in range(limit):
            r.append(currentRound - (limit-l))
        return r


def findLeagueSeasonTeamRecentPreviousRounds(data, league, season, team, round):
    rounds = findRecentPreviousRounds(round, 6)         # by definition is 6, can change for optimization
    if rounds is None:
        return None

    previous_matches_pd =  data[(data["League"]==league) & ((data["Home_Team"]==team) | (data["Away_Team"]==team)) & (data["Season"]==season) & (data["Round"].isin(rounds))]
    recent_perf = 0
    for index, row in previous_matches_pd.iterrows():
        hteam = row['Home_Team']
        ateam = row['Away_Team']
        if hteam==team:
            recent_perf = recent_perf + (row['Home_Score']-row['Away_Score'])
        else:
            recent_perf = recent_perf + (row['Away_Score']-row['Home_Score'])

    return recent_perf


def fillWithRecentPerformance(record, data):
    # get home team and away team and round
    league = record['League']
    season = record['Season']
    round = record['Round']
    hteam = record['Home_Team']
    ateam = record['Away_Team']
    
    home_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, hteam, round)
    away_team_goal_diff = findLeagueSeasonTeamRecentPreviousRounds(data, league, season, ateam, round)

    return [home_team_goal_diff, away_team_goal_diff]

In [34]:
# load all directory as league name list
dir = "./Results"
leagues = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

# loop to open csv
result_with_goal_sofar_pd = pd.DataFrame()
for league in leagues:
    print("process league: " + league + "...")
    league_folder = os.path.join(dir, league)
    csv_file_for_league = [os.path.join(league_folder, name) for name in os.listdir(league_folder) if name.endswith('.csv')]
    
    for csv_filename in csv_file_for_league:
        current_league_season_pd = pd.read_csv(csv_filename, skiprows=[0], names=["Home_Team", "Away_Team", "Result", "Link", "Season", "Round", "League"])

        # Divide result into home_score and away_score
        df_score =  current_league_season_pd['Result'].str.extract(r'(\d)-(\d)')
        current_league_season_pd.insert(loc=3, column="Home_Score", value=df_score[0].astype('Int64'))     # use Int64 as it support NaN
        current_league_season_pd.insert(loc=4, column="Away_Score", value=df_score[1].astype('Int64')) 

        if len(current_league_season_pd)>0:
            # get home team and away team total goal so far
            home_away_total_goal_sofar = current_league_season_pd.apply(fillWithTotalGoalSoFar, data=current_league_season_pd, axis=1)
            goal_so_far_list = np.array(home_away_total_goal_sofar.values.tolist())         # convert to list
            home_away_total_goal_sofar_pd = pd.DataFrame(goal_so_far_list, columns=["HOME_GOAL_SO_FAR", "AWAY_GOAL_SO_FAR"])    # convert to dataframe
            current_league_season_pd.insert(loc=5, column="HOME_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["HOME_GOAL_SO_FAR"]) 
            current_league_season_pd.insert(loc=6, column="AWAY_TOTAL_GOAL_SO_FAR", value=home_away_total_goal_sofar_pd["AWAY_GOAL_SO_FAR"])     

            # merge with ELO
            result_elo_pd = current_league_season_pd['Link'].apply(fillWithELO)   
            elo_list = np.array(result_elo_pd.values.tolist())
            elo_df = pd.DataFrame(elo_list, columns=["ELO_HOME", "ELO_AWAY"])
            current_league_season_pd.insert(loc=8, column="ELO_HOME", value=elo_df["ELO_HOME"]) 
            current_league_season_pd.insert(loc=9, column="ELO_AWAY", value=elo_df["ELO_AWAY"]) 

            # get recent performance
            home_away_recent_perf = current_league_season_pd.apply(fillWithRecentPerformance, data=current_league_season_pd, axis=1)
            perf_list = np.array(home_away_recent_perf.values.tolist())
            home_away_perf_pd = pd.DataFrame(perf_list, columns=["HOME_LAST_6_GOAL_DIFF", "AWAY_LAST_6_GOAL_DIFF"])
            current_league_season_pd.insert(loc=7, column="HOME_LAST_6_GOAL_DIFF", value=home_away_perf_pd["HOME_LAST_6_GOAL_DIFF"]) 
            current_league_season_pd.insert(loc=8, column="AWAY_LAST_6_GOAL_DIFF", value=home_away_perf_pd["AWAY_LAST_6_GOAL_DIFF"]) 

            result_with_goal_sofar_pd = pd.concat([result_with_goal_sofar_pd, current_league_season_pd])

# export to csv
result_with_goal_sofar_pd.to_csv('cleaned_dataset.csv', index=False)

process league: championship...
process league: eerste_divisie...
process league: primeira_liga...
process league: ligue_1...


In [24]:
result_with_goal_sofar_pd.head(30)

,Home_Team,Away_Team,Result,Home_Score,Away_Score,HOME_TOTAL_GOAL_SO_FAR,AWAY_TOTAL_GOAL_SO_FAR,HOME_LAST_6_GOAL_DIFF,AWAY_LAST_6_GOAL_DIFF,Link,ELO_HOME,ELO_AWAY,Season,Round,League
0,Watford,Middlesbrough,1-0,1,0,0,0,None,None,https://www.besoccer.com/match/watford-fc/midd...,65.0,60.0,2021,1,championship
1,Birmingham City,Brentford,1-0,1,0,0,0,None,None,https://www.besoccer.com/match/birmingham-city...,52.0,59.0,2021,1,championship
2,Wycombe Wanderers,Rotherham United,0-1,0,1,0,0,None,None,https://www.besoccer.com/match/wycombe-wandere...,41.0,48.0,2021,1,championship
3,AFC Bournemouth,Blackburn Rovers,3-2,3,2,0,0,None,None,https://www.besoccer.com/match/afc-bournemouth...,63.0,57.0,2021,1,championship
4,Barnsley,Luton Town,0-1,0,1,0,0,None,None,https://www.besoccer.com/match/barnsley-fc/lut...,47.0,50.0,2021,1,championship
5,Bristol City,Coventry City,2-1,2,1,0,0,None,None,https://www.besoccer.com/match/bristol-city-fc...,58.0,45.0,2021,1,championship
6,Cardiff City,Sheffield Wednesday,0-2,0,2,0,0,None,None,https://www.besoccer.com/match/cardiff-city-fc...,60.0,60.0,2021,1,championship
7,Derby County,Reading,0-2,0,2,0,0,None,None,https://www.besoccer.com/match/derby-county-fc...,61.0,54.0,2021,1,championship
8,Huddersfield Town,Norwich City,0-1,0,1,0,0,None,None,https://www.besoccer.com/match/huddersfield-to...,57.0,61.0,2021,1,championship
9,Millwall,Stoke City,0-0,0,0,0,0,None,None,https://www.besoccer.com/match/millwall-fc/sto...,55.0,65.0,2021,1,championship
